# Homework 3 - What is the best anime in the world?

## 1. Data collection

In [122]:
import requests as r
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
from datetime import datetime

### 1.1. Get the list of animes

In [21]:
# 19130 anime in total, 383 pages with 50 animes

with open("C:/Users/andre/Desktop/list_anime.txt", "w", encoding='utf-8') as file:
    
    for page in tqdm(range(0, 383)):
    
        # URL of the single page 
        url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
        
        # Get response object
        response = r.get(url)
        
        # Get html
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all links iterating through the document
        for tag in soup.find_all("tr", class_="ranking-list"):
            a_list = tag.find_all('a', class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True)
            for a in a_list:
                link = a['href']
                file.write(str(link) + '\n')
                
file.close()

100%|████████████████████████████████████████████████████████████████████████████████| 383/383 [05:12<00:00,  1.23it/s]


### 1.2. Crawl animes

In [33]:
file = open("C:/Users/andre/Desktop/prova_anime.txt", "r", encoding='utf-8')

counter_anime = 0
page = 0

for line in file:
    counter_anime += 1
    if (counter%50 == 1):
        page +=1
    response = r.get(line)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    try:
        os.mkdir("C:/Users/andre/Desktop/Anime/Page" + str(page))
    except OSError:
        pass
    
    with open("C:/Users/andre/Desktop/Anime/Page" + str(page) + "/article_" + str(counter_anime)+ ".html", "w", encoding='utf-8') as file:
        file.write(str(soup))

In [24]:
# Count anime links in file

file = open("C:/Users/andre/Desktop/list_anime.txt", "r", encoding='utf-8')
line_count = 0
for line in file:
    if line != "\n":
        line_count += 1
file.close()

print(line_count)

19130


### 1.3 Parse downloaded pages

In [279]:
# Create file .tsv

with open("C:/Users/andre/Desktop/Anime/Page1/anime1.html", "r", encoding='utf-8') as file:
    
    # Read file and parse html code
    content_page = file.read()
    soup = BeautifulSoup(content_page, 'html.parser')
    
    title = soup.find_all("h1", class_="title-name h1_bold_none")[0].find("strong").text
    
    information = soup.find_all("div", class_="spaceit_pad")
    
    # Iterate over information ResultSet
    for i in range(0, len(information)):
        
        # Anime type
        if information[i].find("span", string='Type:') != None:
            animeType = information[i].find('a').text
            
        # N-episodes
        if information[i].find("span", string='Episodes:') != None:
            animeNumEpisode = information[i].contents[2].strip()
        
        # Dates
        if information[i].find("span", string='Aired:') != None:
            dates = information[i].contents[2].strip().split("to")
            releaseDate = datetime.strptime(dates[0].strip(), '%b %d, %Y').date()
            endDate = datetime.strptime(dates[1].strip(), '%b %d, %Y').date()
            
        # Number of members
        if information[i].find("span", string='Members:') != None:
            animeNumMembers = information[i].contents[2].strip()
        
        # Score of anime
        if information[i].find("span", itemprop='ratingValue') != None:
            animeScore = information[i].find("span", itemprop='ratingValue').text
            
        # Anime Users
        if information[i].find("span", itemprop='ratingCount') != None:
            animeUsers = information[i].find("span", itemprop='ratingCount').text
        
        # Anime Rank
        if information[i].find("span", string='Ranked:') != None:
            animeRank = information[i].contents[2].strip().replace("#", "")
            
        # Anime Popularity
        if information[i].find("span", string='Popularity:') != None:
            animePopularity = information[i].contents[2].strip().replace("#", "")
    
    # Anime Description
    animeDescription = soup.find("p", itemprop='description').text
    
    # Related Animes
    related_anime_table = soup.find("table", class_="anime_detail_related_anime")
    links_list = [link.text for link in related_anime_table.find_all("a")]
    links_list = list(set(links_list))
    
    # Characters
    characters_div = soup.find_all("div", class_="detail-characters-list clearfix")[0]
    
    animeCharacters = characters_div.find_all("h3", class_="h3_characters_voice_actors")
    animeCharacters = [character.text for character in animeCharacters]
    
    animeVoices = characters_div.find_all("td", class_="va-t ar pl4 pr4")
    animeVoices = [actor.find("a").text for actor in animeVoices]
    
    div = soup.find_all("div", class_="detail-characters-list clearfix")[1]
    tds = div.find_all("td", class_="borderClass")
    a = []
    for td in tds:
        print(td.find_all("a", class_="fw-n"))
    
    small = div.find_all("small")
    lista = zip(a, small)
        
file.close()

[<a class="fw-n" href="https://myanimelist.net/people/203/Justin_Cook">
<img alt="Cook, Justin" class="lazyload" data-src="https://cdn.myanimelist.net/r/42x62/images/voiceactors/2/12472.jpg?s=022c537cbbc0d397a3403db7bced4c3e" data-srcset="https://cdn.myanimelist.net/r/42x62/images/voiceactors/2/12472.jpg?s=022c537cbbc0d397a3403db7bced4c3e 1x, https://cdn.myanimelist.net/r/84x124/images/voiceactors/2/12472.jpg?s=51263c7d2af02452fe3376ef4d372f8b 2x" height="62" width="42"/>
</a>]
[]
[<a class="fw-n" href="https://myanimelist.net/people/55215/Noritomo_Yonai">
<img alt="Yonai, Noritomo" class="lazyload" data-src="https://cdn.myanimelist.net/r/42x62/images/questionmark_23.gif?s=f7dcbc4a4603d18356d3dfef8abd655c" data-srcset="https://cdn.myanimelist.net/r/42x62/images/questionmark_23.gif?s=f7dcbc4a4603d18356d3dfef8abd655c 1x, https://cdn.myanimelist.net/r/84x124/images/questionmark_23.gif?s=b2caae0942e99347ceebbfe730682689 2x" height="62" width="42"/>
</a>]
[]
[<a class="fw-n" href="https://m